# Recherche d'Information et traitement de données massives

# Lab 4 : Recherche d’information sur le Web 


L'objectif de cette séance est d'étudier le problème de la recherche d'information sur le WEB. Elle constiendra deux parties.
 + La première partie s'intéresse aux approches permettant le passage à l'échelle des données du WEB et notamment au paradigme MapReduce. Vous pratiquerez notamment dans cette partie l'écriture d'algorithmes selon le cadre MapReduce.
 + La deuxième partie s'intéresse à la prise en compte de la structure de la collection du WEB au travers de la mise en oeuvre d'approches d'analyse de liens pour l'ordonnancement.
 + Plusieurs exercices d'approfondissement facultatifs vous sont aussi proposés.

## PARTIE 1 :  RECHERCHE WEB et PASSAGE À l'ECHELLE
## EXERCICES : écriture d'algorithmes en MapReduce (sur papier)

### Avant propos : un bref rappel de MapReduce
Comme nous l'avons vu dans le cours 4, Map Reduce est un **modèle de programmation** (ou patron de programmation) qui fournit un cadre pour automatiser le calcul distribué sur des données massives. Ce cadre propose d'écrire tout traitement à l'aide de deux opérations `map` et  `reduce` et de la représentation des données sous la forme de paires `(clé,valeur)`.
MapReduce est aussi un **framework d'exécution** qui permet l'éxécution distribuée des programmes écrits selon ce cadre, et cela de manière totalement transparente selon le schéma rappelé ci-dessous.


<img src="./Figures/map-reduce.png" width="500" height="500" />

1.  Un certain nombre de tâches *Map* sont alimentées par une ou
    plusieurs partitions de données en provenance d'un système de
    fichiers distribués (par exemple GFS, HDFS, S3).  Ces tâches *Map* transforment ces données en une séquence de paires clés-valeurs. C'est le développeur qui détermine comment sont calculées les paires
    clés-valeurs en fonction des données en entrée en écrivant le code dans la fonction `map()`.
2.  Les paires clés-valeurs sont collectées par un contrôleur maître et triées par clés.
    Les paires sont redirigées vers les tâches *Reduce* de façon à ce que toutes les paires
    qui ont la même clé soient redirigées vers la même tâche *Reduce*.
3.  Les tâches *Reduce* traitent les clés une par une. Elles agrègent/combinent les valeurs associées
    aux clés selon le code spécifié dans la fonction `reduce()`.



Dans le cours, nous avons vu comment écrire en MapReduce un programme permettant de compter le nombre d'occurences des mots d'une collection donnée (programme **WordCount**). C'est une tâche qui peut sembler très simple à écrire quand on travaille sur une collection de petite taille. Vous avez d'ailleurs proposé une solution non-distribuée pour cette tâche pour la collection TIME (étape de filtrage par mots fréquents ou calcul de la pondération TF). En python, nous pouvons écrire cela très simplement, par exemple comme dans le programme ci-dessous.


In [0]:
def count_frequency(collection):
    tokens_count={}
    for doc_id in collection:
        for token in collection[doc_id]:
            if token in tokens_count.keys():
                tokens_count[token] +=1
            else:
                tokens_count[token]=+1
    return tokens_count

Dans le cas où l'on considère une grosse collection dont le stokage et les traitements nécéssitent d'être distribués, alors cette tâche devient plus diffile à écrire, notamment car il faut prendre en compte la distribution des données et des traitements. Le modèle MapReduce apporte une solution à cela en proposant d'écrire ce type de programme selon le principe suivant :

+ On considère que le document ou la partie du document est donné sous la forme d'une paire (clé,valeur) avec comme clé, l'identifiant du document et comme valeur le contenu textuel du document.

+ Étant donnée une collection d’items, appliquer à chaque item un processus de transformation individuelle (étape `MAP`) qui produit des valeurs intermédiaires étiquetées. Dans le cas du WordCount, il s'agit juste de prendre chaque token du document ou de la partie du document et de la transformer en la paire (mot,1) comme illustré ci-dessous.

<img src="./Figures/diapositive12.jpg" width="500" height="500" />


+ Regrouper ces valeurs intermédiaires par étiquette (étape faite par le framework `SHUFFLE AND SORT`). On aura dans le cas du WordCount en sortie de cette étape en ensemble de paires (mot, [1,1,1,1..]) avec comme clés les différents mots du documents et comme valeur une liste des 1-occurence des mots dans le document considéré.
+ Appliquer une fonction d'agrégation à chaque groupe (étape `REDUCE`).Dans le cas du Wordcount il s'agit juste de sommer la liste des occurences comme illustré ci-dessous.

<img src="./Figures/diapositive15.jpg" width="500" height="500" />


### Concevoir un programme selon le cadre MapReduce : un peu de méthodologie

Pour faciliter l'écriture de programmes selon le cadre MapReduce, il est souvent nécessaire de se poser d'abord  les questions ci-dessous.

 + De quelle nature sont les documents en entrée ? Comment les représenter sous une forme `(clé, valeur)` ?
 + Quelles sont les groupes visés ? Quelles sont les valeurs intermédiaires que je cherche à produire depuis mon document d'entrée ?
 + Quelle est la valeur finale ? Quelle est la nature de l'agrégation pour produire cette valeur finale ? 

Une fois les réponses à ces questions claires, il suffira ensuite :
 + D'écrire la fonction de `map` qui prend en entrée un document et qui produit une séquence de paires `(clé, valeur)`.
 + D'avoir en tête que ces différentes paires sont collectées par le framework et triées par clés pour donner une entrée à la tâche d'agrégation l'ensemble des paires qui ont la même clé. 
 + D'écrire la fonction `reduce` qui prend en entrée une paire `(clé, liste(valeurs))` en spécifiant le traitement d'agrégation voulu. 

Le schéma d'éxécution et sa trace sur le problème du WordCount vous est donné ci-dessous, pour rappel.

<img src="./Figures/diapositive16.jpg" width="500" height="500" />

<img src="./Figures/diapositive17.jpg" width="500" height="500" />

Nous allons maintenant appliquer le cadre MapReduce au calcul de la pondération `TF-IDF`.

### A votre tour : Tf-IDF en MapReduce

L'objectif est de calculer `Tf-IDF` pour un ensemble de documents en utilisant le modèle MapReduce. Pour rappel, comme vu dans le cours 1, `Tf-IDF (Term frequency-Inverse Document Frequency)` est une statistique qui traduit le niveau d'importance d'un terme $t$ pour un document $d$ appartenant à une collection (ou un corpus) de taille $N$. Dans cet exercice, on considèrera la formulation mathématique suivante :

$$TF-IDF(t_i,d) = ( \frac{tf_{t_i,d}}{\sum_{t_k \in d} tf_{t_k,d}}  ) \times \log \left( \frac{N}{df_t}\right)$$

où $tf_{t,d}$ est le nombre d'occurrence du terme $t$ dans le document $d$, $N$ le nombre de documents de la collection et $df_t$ le nombre de documents dans lesquels $t$ est présent.

Vous pouvez prendre le temps de regarder la correction du Lab2 dans lequel nous avons proposé des solutions pour calculer cette statistique sur la collection TIME.

Pour calculer cette statistique sur une très grosse collection de documents, il est nécéssaire de distribuer son calcul. Nous allons pour cela découper le travail en 3 étapes :

 + **1- Le calcul, pour chaque mot, de son nombre d'occurences par document.** Nous appelerons cette étape `WordFrequenceInDocs`.
 + **2- Le calcul du nombre de mots par documents**. Nous appelerons cette étape `WordCountsForDocs`.
 + **3- La combinaison des 2 informations précédentes pour calculer le TF_IDF**. Nous appelerons cette étape `WordsInCorpusTFIDF`.
 
#### Etape 1 : WordFrequenceInDocs

Il s'agit donc ici de formuler le problème du calcul du nombre d'occurences des mots d'un document avec le cadre MapReduce. C'est un problème très proche du problème du WordCount vu en cours et rappelé ci-dessus et vous devriez pouvoir y répondre rapidement.

**Données d'entrée**

Votre premier travail est de proposer une représentation adéquate de vos données d'entrée (une collection de document) pour le cadre MapReduce. On considère que chaque tâche `MAP` traitera un seul document (ou partie de document) et c'est donc ce document (ou cette partie) qui sera pris comme entrée de la fonction `MAP`.
Que proposez-vous ?

# A completer
Un dictionnaire dont la clé est l'identifiant du document et le valeur est le document en soit (une string de texte par exemple)


**Fonction MAP**

En vous inspirant du WordCount, écrire en pseudo-code, la fonction MAP pour cette étape. Attention, il faudra pouvoir garder l'information relative à l'identifiant du document considéré.


In [4]:
# A completer
def my_map((doc_id, doc_content)):
    tokens_count = []
    for token in doc_content:
        tokens_count.append((token, doc_id), 1)
    return tokens_count



L'ensemble des paires (clé,valeur) provenant des différents noeuds `MAP` sont collectées et triées par clés intermédiaires et donc fournies sous cette forme à la tâche `REDUCE`. Ecrire, en pseudo-code la fonction `REDUCE` pour `WordFrequenceInDocs`.



In [7]:
# A completer
def my_reduce(tuple_word_doc_id, list_of_ones): #tuple_word_doc_id = (word, doc_id)
    return (tuple_word_doc_id, len(list_of_ones)) #returns the term frequency of word in doc_id
    
    


#### Etape 2: WordCountsForDocs

Il s'agit maintenant de calculer le nombre de mots par documents. Deux petites indications :
+ L'entrée de cette tâche sera la sortie de la tâche précédente soit une paire de type 
`((word,doc_id),n)` avec `n` le nombre d'occurence du terme word dans le document `doc_id` soit le tf.
+ Pour cette tâche, il pourrait être intéressant de pouvoir avoir accès aux documents par le mécanisme (clé,valeur).

Ecrire, en pseudo-code, la fonction `MAP`, puis la fonction `REDUCE` vous permettant de faire cette étape.

In [11]:
# A compléter
#Map:
def map_WordCountsForDocs(tuple,n): #tuple = (word,doc_id)
    my_dict = {}
    my_dict[doc_id] = n
    return my_dict

#Reduce:
def reduce_WordCountsForDocs(doc_id, list_of_counts):
    return (doc_id, sum(list_of_counts)) #return the number of words in doc_id

    


#### Etape 3: WordsInCorpusTFIDF

Il s'agit maintenant de combiner les informations précédentes pour calculer le TF-IDF pour chaque terme. On considère à nouveau ici que l'entrée est la sortie de la tâche précédente soit une paire de type 
`((word,doc_id),n/N)` avec `n` le nombre d'occurence du terme word dans le document `doc_id` et `N` le nombre total de mots dans le document `doc_id`. On supposera aussi que le nombre de documents $D$ dans la collection est passé au système sous la forme d'une constante.

Ecrire, en pseudo-code, la fonction `MAP`, puis la fonction `REDUCE` vous permettant de faire cette étape.


In [16]:
# A compléter
def map_WordsInCorpusTFIDF( tuple, r): #tuple = (word,doc_id), r = n/N
    
    return 

### A la maison :  Construction d'un index inversé en MapReduce

Il s'agit ici de refléchir à comment un index inversé de documents peut être construit de manière distribuée à l'aide de MapReduce. 

1. Ecrivez en pseudo-code la fonction `MAP` en précisant bien le type des données d'entrée.



In [0]:
# A completer

2. Ecrivez en pseudo-code la fonction `REDUCE`.

In [0]:
# A compléter

3. Un test sur machine

Pour vous permettre de mettre en oeuvre ce mécanisme de manière concrète, vous allez appliquer cela à l'indexation d'une collection [books.json](../Data/books.json).
On cherche à produire un fichier inversé qui indique pour chaque mot, la liste des livres dans lesquels il apparaît en utilisant le cadre MapReduce.

Pour cela, nous vous fournissons dans le répertoire [Utils](./Utils) un fichier [Lab4.py](./Utils/Lab4.py) qui contient un ensemble de fonctions qui vous seront utiles.




Importer les fonctions utiles de ce module python à l'aide de la commande ci-dessous.

In [0]:
from Utils.Lab4 import *

A l'aide de la fonction `readData(filename)` de ce module, charger le fichier [books.json](../Data/books.json).

Ecrire la fonction `mapper` nécessaire à la construction de l'index inversé.

In [0]:
def mapper(data):
    # A completer

Ecrire la fonction `reducer` nécessaire à la construction de l'index inversé.

In [0]:
def reducer(data): 
    # A completer

Tester vos deux fonctions en appliquant le code ci-dessous

In [0]:
def invertedIndexExample(filename):
    m = MapReduce(mapper, reducer)
    results = m(readData(filename))
    for w, b in results:
        print("mot : ", w, "livres : ", b)
    return
    
invertedIndexExample('./Data/books.json')

## PARTIE 2 : Algorithme de PageRank

Dans cette partie nous allons implémenter et étudier l'algorithme d'ordonnancement PageRank. Ce dernier est utilisé par le moteur de recherche Google pour attribuer un score d'importance à chaque page Web. D'ailleurs, c'est en partie sur la base de ce score (et de nombreux autres facteurs) que le moteur de recherche classe par ordre d'importance les pages Web correspondant à une recherche donnée.


### Le format des données

Nous allons utiliser le format de type "graphe" pour représenter un nombre $N$ de page Web. Voici un exemple de graphe :

<img src="./Figures/graph.png" width="500" height="500" />

La première ligne indique le nombre de pages Web dont il est question pour le graphe. Puis pour chacune des lignes suivantes le premier numéro définit l'identifiant d'une page et les numéros suivants les liens sortants de cette page. 

Vous disposez dans le dossier [Data](./Data) associé à ce Lab de plusieurs graphes de tailles différentes.

Comme nous allons manipuler des matrices dans cette partie, nous utiliserons la bibliothèque numpy que vous pouvez installer avec la commande ci-dessous.

In [17]:
import numpy as np

#### Première étape : Matrice d'Adjacence.

Nous nommons $A$ matrice d'adjacence, la matrice de dimension $N \times N$ telle que $a_{i,j}=1$ s'il existe un lien de la page $i$ vers la page $j$, et $0$ sinon.


**1. Ecrire une fonction `build_adjacency_matrix(filename)` permettant de construire la matrice d'adjacence à partir d'un graphe dont le format est défini plus haut (et au format .txt).**





In [30]:
with open('./Data/graph1.txt', 'r') as f:
    N = f.readline()
    line = f.readline().split()
    print(line)

['0', '2', '3']


In [41]:
adjacency_matrix = np.zeros((2,2))
adjacency_matrix
adjacency_matrix[0,0] = 1
adjacency_matrix

array([[1., 0.],
       [0., 0.]])

In [54]:
import numpy as np

def build_adjacendy_matrix(filename):
    """Build the adjacency matrix from the graph file"""
    with open(filename, 'r') as f:
        N = int(f.readline())
        adjacency_matrix = np.zeros((N,N))
        for _ in range(N):
            line = f.readline().split()
            depart = int(line.pop(0))
            for column in line:
                adjacency_matrix[depart,int(column)] = 1
        return adjacency_matrix


**2. Ouvrir le graph1.txt présent dans le dossier Data et dessiner le à la main avec des noeuds portant les numéros de pages et des flèches représentant les liens.**




**3. Appliquer la fonction `build_adjacency_matrix()` au graph1.txt présent dans le dossier Data.**


In [56]:
build_adjacendy_matrix('./Data/graph1.txt')# A completer

array([[0., 0., 1., 1., 0., 0.],
       [1., 0., 0., 1., 0., 1.],
       [0., 1., 0., 1., 1., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 1., 0., 0.]])

#### Deuxième  étape : Matrice de transition.

A partir de la matrice d'adjacence, il est possible de calculer une matrice de transition $P$ de paramètre $\lambda$ dont nous aurons besoin lors de l'implémentation de l'algorithme de PageRank. La matrice de transition est définie par :

$$ P_{i,j} =  \left\{
\begin{array}{l}
   \left.\begin{array}{l} 
   \lambda \frac{a_{i,j}}{\sum_{j=1}^N a_{i,j}} + (1-\lambda ) \frac{1}{N}  \text{ si} \sum_{j=1}^N a_{i,j} \neq 0\\
   \frac{1}{N} \text{ sinon.}\\
   \end{array}\right.  \\
\end{array}
\right.$$


**4.  Ecrire une fonction `build_transition_matrix(filename, lam)` permettant de construire la matrice de transition à partir d'un graph (au format .txt) et d'une valeur pour le paramètre $\lambda$ (correspondant à lam car lambda est un mot clé du langage python).**

In [83]:
x = np.zeros((3,3))
x
len(x)
list(range(3))

[0, 1, 2]

In [75]:
x[0,:] = 1
x
x[0,:] = x[0,:]/2
x[0,:] = x[0,:]*10 + 30
x


array([[35., 35., 35.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [97]:
import numpy as np

def build_transition_matrix(filename, lam):
    """Build the transition matrix from the adjacency matrix"""
    # A completer
    adjacency_matrix = build_adjacendy_matrix(filename)
    N = len(adjacency_matrix)
    for line in range(N):
        somme = sum(adjacency_matrix[line,:])
        if somme != 0:
            adjacency_matrix[line,:] = (lam/somme)*adjacency_matrix[line,:] + (1-lam)/N
        else:
            adjacency_matrix[line,:] = 1/N
        
    return adjacency_matrix
    


**5. Appliquer cette fonction au graph1.txt présent dans le dossier Data pour une valeur de lambda de 0.85 (valeur habituellement choisie).**

In [98]:
build_transition_matrix(('./Data/graph1.txt'), 0.85)

array([[0.025     , 0.025     , 0.45      , 0.45      , 0.025     ,
        0.025     ],
       [0.30833333, 0.025     , 0.025     , 0.30833333, 0.025     ,
        0.30833333],
       [0.025     , 0.2375    , 0.025     , 0.2375    , 0.2375    ,
        0.2375    ],
       [0.875     , 0.025     , 0.025     , 0.025     , 0.025     ,
        0.025     ],
       [0.025     , 0.025     , 0.025     , 0.875     , 0.025     ,
        0.025     ],
       [0.025     , 0.025     , 0.45      , 0.45      , 0.025     ,
        0.025     ]])

**6. Appliquer la fonction somme sur les colonnes de la matrice de transition $P$ (sommes de tous les éléments d'une ligne, ligne par ligne). Que constatez-vous ?**

In [100]:
# A completer
transition_matrix = build_transition_matrix(('./Data/graph1.txt'), 0.85)
transition_matrix.sum(1)

array([1., 1., 1., 1., 1., 1.])

**Interpretation de la matrice de transition**

Le PageRank simule le chemin aléatoire d'un internaute qui naviguerait sur le Web en choisissant au hasard les liens à suivre sur une page donnée. Par exemple, dans le cas de la figure plus haut (en section intitulée "le format des données"), un internaute situé en page 1 choisirait d'aller à la page 0, 3 ou 2 avec probabilité 1/3, 1/3, 1/3 car il y a trois liens sortants de la page 1. Les éléments de la matrice de transition représentent en effet des probabilités de transition à chaque nœud qui peuvent être décrites globalement par la matrice de transition $P$ de dimension $N\times N$, où $N$ est le nombre de nœuds dans le graphique et l'élément $p_{𝑖, 𝑗}$ est la probabilité qu'un internaute aléatoire passe de la page $i$ à la page $j$.


**7. Dans le cas du graph1.txt, selon votre lecture visuelle du graph1 quelles sont les probabilités d'un internaute situé en page 5 d'aller dans chacunes des autres pages web ? Et selon votre matrice de transition, quelles sont ces mêmes probabilités ? Que constatez vous ?**




In [101]:
# A compléter
# He can go to either page 2 or 3 with probability 1/2. In the transition matrix we take into account the probability
# that the user "jumps" to any other page with a small probability 0,025.

**8. Calculer à nouveau la matrice de transition avec $\lambda=1$. Que constatez-vous ? Nous reviendrons sur l'étude du paramètre $\lambda$ et de son rôle par la suite.**


In [110]:
transition_matrix = build_transition_matrix(('./Data/graph1.txt'), 1)
transition_matrix


array([[0.        , 0.        , 0.5       , 0.5       , 0.        ,
        0.        ],
       [0.33333333, 0.        , 0.        , 0.33333333, 0.        ,
        0.33333333],
       [0.        , 0.25      , 0.        , 0.25      , 0.25      ,
        0.25      ],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.5       , 0.5       , 0.        ,
        0.        ]])

En parcourant plusieurs fois des balades aléatoires sur le graphe, PageRank simule le comportement de plusieurs internautes aléatoires. Les pages qui reçoivent un plus grand nombre de visites sont considérées comme plus importantes que celles qui ne le sont que rarement. 


Notons qu'au départ, quand un internaute commence sa promenade, il peut être n'importe où dans le graphe. Si nous n'avons aucune raison de penser qu'il serait plus susceptible de choisir une page plus qu'une autre comme point de départ, nous pouvons dire que la probabilité initiale que l'internaute se trouve sur une certaine page est égale à $\frac{1}{N}$.

Ainsi, au départ, la distribution de probabilité de la position de l'internaute peut être décrite par un vecteur colonne $R^{(0)}$ avec $N$ éléments prenant pour valeur $\frac{1}{N}$. Ce vecteur s'appelle le vecteur PageRank à l'itération initiale $l=0$. Pour une itération quelconque nous le noterons $l$.


#### Troisième  étape : Algorithme de PageRank

Nous avons à ce moment en notre possession l'ensemble des éléments et outils pour implémenter facilement l'algorithme de PageRank suivant :

<img src="./Figures/Algo_PR.png" width="700" height="700" />

**9. Implémenter l'Algorithme 1 par une fonction `page_rank(transition_matrix, epsilon = 1e-3):` en réutilisant toutes les fonctions que vous venez de construire plus haut et en fixant tout d'abord $\lambda=0.85$. Attention : dans la partie initialisation, pensez à initialiser l'erreur $\epsilon$, par exemple à la valeur $1$.**

In [120]:
x = np.array([3,4])
x 
np.linalg.norm(x)
N = 6
R_1 = np.full((N, 1), 1/N)
R_1

array([[0.16666667],
       [0.16666667],
       [0.16666667],
       [0.16666667],
       [0.16666667],
       [0.16666667]])

In [150]:
def page_rank(transition_matrix, epsilon=1):
    """Run the PageRank algorithm based on the transition matrix"""
    N = len(transition_matrix)
    R_1 = np.full((1, N), 1/N)
    R_2 = np.dot(R_1, transition_matrix)
    l = 0
    while np.linalg.norm(R_2 - R_1) > epsilon:
        l += 1
        R_1 , R_2 = R_2, np.dot(R_2, transition_matrix)
    #print(np.sum(R_1))
    print('number of iterations for epsilon = {} : {}'.format(epsilon, l))
    return R_1





**10. Appliquez l'algorithme 1 au graph1.txt avec $\lambda=0.85$ et $\epsilon = 10^{-3}$**

In [154]:
page_rank(build_transition_matrix(('./Data/graph1.txt'), 0.85), 0.0001)

number of iterations for epsilon = 0.0001 : 20


array([[0.29902267, 0.06483634, 0.18740393, 0.30069957, 0.06483634,
        0.08320116]])

#### Quatrième étape : Etude et analyse de l'algorithme de PageRank implémenté.

Rappel : fixez tout d'abord $\lambda=0.85$.

**11. Choisissez différentes valeurs pour le critère d'arrêt telles que $\epsilon = {10^{-3}, 10^{-4}, 10^{-5}}$. Qu'est-ce que vous observez ?**












In [149]:
# A compléter
print('For e = 10-3:', page_rank(build_transition_matrix(('./Data/graph1.txt'), 0.85), 0.001))
print('For e = 10-4:', page_rank(build_transition_matrix(('./Data/graph1.txt'), 0.85), 0.0001))
print('For e = 10-5:',  page_rank(build_transition_matrix(('./Data/graph1.txt'), 0.85), 0.00001))

For e = 10-3: [[0.29935795 0.06491206 0.18716453 0.30040839 0.06491206 0.08324501]]
For e = 10-4: [[0.29902267 0.06483634 0.18740393 0.30069957 0.06483634 0.08320116]]
For e = 10-5: [[0.29899138 0.06482929 0.18742625 0.3007267  0.06482929 0.08319708]]


**12. Ajouter quelques hubs (pages qui ont beaucoup de liens sortant) et autorités (pages qui ont beaucoup de liens entrant). Quelles pages sont classées le plus haut ?**

In [0]:
# A compléter

**13. Essayez d’accroître les rangs de certaines pages. Expliquez votre méthode et validez-la expérimentalement.**

In [0]:
# A compléter

**14. Essayez différentes valeurs pour le facteur d’amortissement $\lambda$. Quel est le comportement de l’algorithme lorsque $\lambda$ tend vers 0 ?**

**15. (À la maison) Amusez vous à tester l'algorithme sur les autres graphes plus volumineux mis à votre disposition dans le dossier Data.**

In [0]:
# A compléter

## PARTIE 3 : PageRank en mode distribué

Nous allons nous intéresser ici au passage à l'échelle du page rank pour permettre de traiter les cas des graphes très volumineux comme le web.

Vous devez avoir bien compris le principe de cet algoritme maintenant et votre premier travail sera donc d'appliquer le cadre MapReduce au calcul du PageRank. Dans un premier temps, vous pourrez ne pas prendre en compte le mécanisme de télétransportation pour vous faciliter la tâche.

**1. Un exemple de graphe**

Pour vous aider, vous pouvez vous aider d'un exemple de graphe simple comme celui ci-dessous. 

<img src="./Figures/graphes_lin.png" width="700" height="700" />

Source : Lin - Data-Intensive Text Processing with MapReduce


A partir de cet exemple, pour vous aider à écrire votre fonction `map`, essayer de réflechir à comment le calcul du page rank transforme les différents noeuds à chaque itération.

**2. Proposez en pseudo code un algorithme Mapeduce pour le calcul du PageRank**

In [0]:
# A completer